<a href="https://colab.research.google.com/github/manuelcozar55/LexAIcon/blob/main/LexAIcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?id=1FBozZB6s5pruTsnXp-SMuhqz4WmQoDIL)
### LexAIcon: Traduce, resume y explica tus textos legales.

Primero se instalan los diferentes elementos necesarios, se realizan los imports y se conecta con la API de Huggingface y a Google Drive

In [2]:
!pip -q install transformers sentencepiece PyPDF2 python-docx torch faiss-cpu huggingface_hub langchain langchain_community accelerate sentence-transformers
!pip -q install -i https://pypi.org/simple/ bitsandbytes

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoConfig, AutoModelForSequenceClassification, BitsAndBytesConfig
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA,LLMChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from PyPDF2 import PdfReader
from docx import Document
import csv
import json
import torch
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from huggingface_hub import login
from google.colab import userdata
from google.colab import drive

token=userdata.get('HF_API')

login(token)

drive.mount('/content/Mydrive/')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Mounted at /content/Mydrive/


A continuación se configura el modelo LLM y al ejecutar se intstalara todo lo necesario para utilizarlo.

In [3]:
# Configurar modelo y tokenizador
model_name = 'Equall/Saul-7B-Instruct-v1'
model_config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


text_generation_pipeline = pipeline(
    model=model_name,
    tokenizer=tokenizer,
    task="text-generation",
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

prompt_template = """
### [INST]
Instrucción: Responda a la pregunta basándose en su conocimiento y en el contexto proporcionado. Asegúrese de que la respuesta sea precisa y detallada, conservando el significado original y utilizando terminología legal adecuada.

### CONTEXTO:
{context}

### PREGUNTA:
{question}
[/INST]
"""

saul_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Crear el prompt desde la plantilla de prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Crear la cadena LLM
llm_chain = LLMChain(llm=saul_llm, prompt=prompt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


En la siguiente sección podemos encontrar todos los compnentes necesarios par la clasificación.

In [52]:
# Cargar modelo clasificador de Huggingface
#def load_classification_model():
 #   tokenizer = AutoTokenizer.from_pretrained("mrm8488/legal-longformer-base-8192-spanish")
  #  model = AutoModelForSequenceClassification.from_pretrained("mrm8488/legal-longformer-base-8192-spanish")
   # return model, tokenizer

# Cargar modelo clasificacdor entrenado
def load_classification_model():
  #Ruta a los archivos de modelo con fine tunning
  model_path = '/content/Mydrive/MyDrive/LexAIcon/MLM_Fine_tunned'
  #Cargar el tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
  #Cargar el modelo usando el archivo safetensors
  model = AutoModelForSequenceClassification.from_pretrained(model_path, from_tf=False, from_flax=False, torch_dtype=None, use_safetensors=True)
  return model, tokenizer

# Cargar modelo de clasificación
classification_model, classification_tokenizer = load_classification_model()

# Establecer etiquetas para cada tipo dentro de la clasificación
id2label = {0: "multas", 1: "politicas_de_privacidad", 2: "contratos", 3: "denuncias", 4: "otros"}

# Función clasificadora
def classify_text(text):
    inputs = classification_tokenizer(text, return_tensors="pt", max_length=4096, truncation=True, padding="max_length")
    classification_model.eval()
    with torch.no_grad():
        outputs = classification_model(**inputs)
    logits = outputs.logits
    predicted_class_id = logits.argmax(dim=-1).item()
    predicted_label = id2label[predicted_class_id]
    return predicted_label

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/Mydrive/MyDrive/LexAIcon/MLM_Fine_tunned and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.

En la siguiente sección tenemos el codigo de las tres funcionalidades, resumen traduccion y explicación que consiste en una clasificación y preguntas a un documento con el contexto mas relevante gracias a la comparacion de embeddings de la query del usuario y el documento de preguntas y respuesta procesado.

In [53]:
# Función para cargar documentos JSON
def load_json_documents(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
    documents = [{"page_content": entry["question"] + " " + entry["answer"]} for entry in json_data["questions_and_answers"]]
    return documents

# Configuración de FAISS (base de datos vectorial) y embeddings
def create_vector_store(docs):
    modelPath = "nlpaueb/legal-bert-base-uncased"
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': False}

    embeddings = HuggingFaceEmbeddings(
        model_name=modelPath,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    split_docs = []
    for doc in docs:
        split_docs.extend([{"page_content": fragment} for fragment in text_splitter.split_text(doc["page_content"])])

    vector_store = FAISS.from_texts([doc["page_content"] for doc in split_docs], embedding=embeddings)
    return vector_store

# Función para traducir texto
def translate(text, target_language):
    context = "Eres un experto traductor de textos legales."
    question = f"Por favor, traduzca el siguiente documento al {target_language} de manera precisa y detallada, asegurándose de conservar el significado original del documento. El texto contiene términos legales que deben ser traducidos con exactitud:\n\n{text}"
    response = llm_chain.invoke({"context": context, "question": question})
    return response

# Función para resumir texto
def summarize(text, summary_length):
    context = "Eres un experto en resumen de textos legales."
    if summary_length == "corto":
        length = "50 palabras"
    elif summary_length == "medio":
        length = "100 palabras"
    elif summary_length == "largo":
        length = "500 palabras"
    else:
        raise ValueError("La longitud del resumen debe ser 'corto', 'medio' o 'largo'.")

    question = f"Por favor, haga un resumen de aproximadamente {length} del siguiente documento legal. Asegúrese de que el resumen sea conciso y que conserve el significado y los detalles críticos del documento original:\n\n{text}"
    response = llm_chain.invoke({"context": context, "question": question})
    return response

# Función para explicación, primero clasifica el texto y luego se pueden hacer preguntas al documento.
# Parámetros son user_input: pregunta del usuario, y document_text: documento introducido por el usuario.
def explain_text(user_input, document_text):
    classification = classify_text(document_text)
    print({"Texto clasificado como: ": classification})

    context = ""
    file_path = None
    if classification == "multas":
        file_path = "/content/Mydrive/MyDrive/LexAIcon/multas_zgz.json"
    elif classification == "denuncias":
        file_path = "/content/MyDrive/MyDrive/LexAIcon/denuncias.json"

    if file_path:
        docs = load_json_documents(file_path)
        if docs:
            vector_store = create_vector_store(docs)
            retriever = vector_store.as_retriever()
            qa = RetrievalQA.from_chain_type(
                llm=saul_llm,
                chain_type="stuff",
                retriever=retriever
            )
            #query_with_context = f"{document_text}\n\nPregunta: {user_input}" Al añadir el documento la informacion de contexto
            #result = qa.invoke(query_with_context)
            result = qa.invoke(user_input)
        else:
            response = "No se encontraron documentos."
    else:
        print("Clasificación no reconocida.")
        result = llm_chain.invoke({"context": document_text, "question": user_input})

    return result

Ahora vamos con la parte de las pruebas a continuación se exponen 4 pruebas completas, una por cada tipo de texto, para entender el funcionamiento de la herramienta

*   Primero inicializamos el texto a introducir, el archivo que traduciremos resumiremos o le preguntaremos cosas.
*   Después se llama a cada una de las funciones con los parámetros correctos y se imprimen sus respuestas.



In [29]:
#Ejemplo multa zona azul
document_text="MULTA POR INCUMPLIMIENTO DE NORMATIVA DE ESTACIONAMIENTO EN ZONA AZUL\n\nFecha de la Infracción: 15 de Mayo de 2024\n\nHora de la Infracción: 10:45 AM\n\nLugar de la Infracción: Calle Alfonso I, 125, Zaragoza, España\n\nNúmero de Expediente: ZA-2024-00567\n\nMatrícula del Vehículo: 1234ABC\n\nDescripción de la Infracción:\n\nEl vehículo con matrícula 1234ABC se encontraba estacionado en una zona de estacionamiento regulado (Zona Azul) sin exhibir el correspondiente ticket de pago visible en el parabrisas. Según la normativa vigente, es obligatorio adquirir y mostrar el ticket de estacionamiento en un lugar visible del vehículo.\n\nNormativa Aplicable:\n\nDe acuerdo con la Ordenanza Municipal de Estacionamiento Regulado de Zaragoza, artículo 5, sección 3, es obligatorio que todos los vehículos estacionados en la Zona Azul dispongan del ticket de estacionamiento correspondiente y que este sea claramente visible.\n\nImporte de la Multa:\n\nEl importe de la multa por esta infracción es de 60 euros.\n\nInstrucciones de Pago:\n\nEl pago de la multa debe realizarse en un plazo de 30 días naturales a partir de la fecha de la notificación de la multa. El pago puede realizarse de las siguientes maneras:\n- En línea a través de la página web del Ayuntamiento de Zaragoza.\n- En cualquier oficina de atención al ciudadano del Ayuntamiento.\n- Mediante transferencia bancaria a la cuenta indicada en la notificación recibida por correo.\n\nDerecho de Alegación:\n\nEl infractor tiene derecho a presentar alegaciones contra esta multa en un plazo de 15 días hábiles a partir de la fecha de la notificación. Las alegaciones deben presentarse por escrito y pueden ser entregadas en cualquiera de las oficinas de atención al ciudadano del Ayuntamiento o enviadas por correo postal a la dirección indicada en la notificación.\n\nNotificación:\n\nLa presente notificación se envía al titular del vehículo registrado en la Dirección General de Tráfico. En caso de que los datos del titular no estén actualizados, se ruega que se actualicen a la mayor brevedad posible para evitar futuras incidencias."

# Ejemplo para Explicar
user_input = "¿Dónde se pueden pagar las multas de tráfico?"
result = explain_text(user_input, document_text)
print(f"Resultado: {result}")

# Ejemplo para Traducir
target_language = "inglés"
result = translate(document_text, target_language)
print(f"Resultado: {result}")

# Ejemplo para Resumir
summary_length = "medio"
result = summarize(document_text, summary_length)
print(f"Resultado: {result}")

{'Texto clasificado como: ': 'multas'}


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Resultado: {'query': '¿Dónde se pueden pagar las multas de tráfico?', 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n¿Dónde se pueden pagar las multas de tráfico en Zaragoza? Las multas de tráfico en Zaragoza se pueden pagar a través de la página web del Ayuntamiento de Zaragoza, en la sección de Trámites y Servicios. También es posible realizar el pago de manera presencial en la Jefatura Provincial de Tráfico de Zaragoza o en las oficinas de atención al ciudadano de la Policía Local.\n\n¿Qué ocurre si no se recibe la notificación de una multa de tráfico? Si no se recibe la notificación de una multa de tráfico, esta se publicará en el sistema TESTRA, y el infractor tendrá 20 días naturales para alegar o pagar la multa con el descuento por pronto pago.\n\n¿Qué pasa si no puedo pagar una multa de tráfico en Zaragoza? Si no puedes pagar una multa de tráfico en 

En este segundo ejemplo introducimos una denuncia como texto de entrada, para poder utilizar el archivo denuncias.json para RAG.

In [ ]:
# Ejemplo denuncia penal
document_text="Denuncia penal Al Juzgado de Instrucción de… que por turno corresponda D/Dª…, [en su caso, en nombre y representación de…] con domicilio a efectos de notificaciones en…, comparezco y como mejor proceda\n\nDIGO\n\nQue por medio del presente escrito y conforme se dispone en los artículos 259 y siguientes de la Ley de Enjuiciamiento Criminal vengo a formular DENUNCIA contra… por un DELITO CONTRA…, tal y como se detalla a continuación.\nLa presente DENUNCIA  se basa en los siguientes:\n\nHECHOS\n\nI.  Que …\n\nII. …\n\nA los expresados hechos le son de aplicación los siguientes,\n\nFUNDAMENTOS DE DERECHO\n\nPrimero.- Que los citados hechos podrían ser constitutivos del delito  tipificado en el artículo… del Código Penal, el cual dispone que…\n\nSegundo.- Que se propone, a los oportunos efectos acreditativos de los hechos anteriormente narrados, la práctica de las siguientes diligencias probatorias:\n1.  Se tengan por acompañados los documentos UNO a CINCO consistentes en…\n2.  Se oficie al Servicio de Protección de la Naturaleza de la Guardia Civil a los efectos de que informe de…\n3.  Se reciba declaración en calidad de imputado a D./Dª…. con domicilio en…\n4.  Se reciba declaración en calidad de testigo a D/Dª… con domicilio en…\n5.  Aquellas otras que se deriven de las anteriores y sirvan para el esclarecimiento de los hechos.\n\nPor lo expuesto,\nAL JUZGADO SOLICITO que teniendo por presentado este escrito, con sus copias se sirva admitir la presente DENUNCIA contra… acordando la incoación de diligencias previas y la averiguación de los hechos que en el presente escrito se relatan al objeto de esclarecer las eventuales responsabilidades penales dimanantes de los mismos.\n\nOTROSÍ DIGO que, siendo intención de este denunciante mostrarse parte en las Diligencias que se instruyesen a raíz del presente escrito de denuncia como acusación particular o popular, interesa a nuestro derecho y así\nSOLICITO que incoado que sea el oportuno procedimiento penal, se cite a esta parte denunciante a fin de ratificar y en su caso ampliar la presente denuncia y se le realice el correspondiente ofrecimiento de acciones conforme dispone el artículo 776 LECrim.\nLugar, fecha y firma.\nRecurriremos a la denuncia penal en lugar de a la administrativa cuando los hechos que pretendamos denunciar puedan ser constitutivos de delito. Es decir, cuando estén tipificados como tal en el Código Penal.\nLos delitos contra el medio ambiente se regulan en el Título XVI del CP, artículos 319 a 340 y los delitos de incendio en los artículos 351 a 358. Te recomendamos la consulta a la “Guía práctica de los delitos ecológicos”, de la colección Cuadernos prácticos de Ecologistas en Acción.\nTambién podemos dirigir nuestra denuncia a la Guardia Civil (especialmente al Servicio de Protección de la Naturaleza –SEPRONA- de la comarca) o a la Fiscalía (lo que puede ser especialmente interesante allí dónde se hayan creado Fiscalías especializadas en medio ambiente y urbanismo).\nLas indicaciones de la denuncia administrativa son igualmente aplicables en este modelo.\nPor ejemplo, “del delito contra la fauna tipificado en el artículo 334.1 del Código Penal”.\nFotografías, videos, croquis, informes, etc. La documentación oportuna de que dispongamos para acreditar los hechos que denunciamos.\nSi lo conocemos, en caso contrario podemos señalar simplemente “cuyos demás datos personales se desconocen” o que “puede ser citado en … que constituye su lugar de trabajo”, por ejemplo.\nSi la denuncia se realiza al Ministerio Fiscal ha de tenerse en cuenta que no existe la posibilidad de personarse en las Diligencias de Investigación Fiscal que éste incoe, pero sí podemos solicitar que “se nos comunique ‘la resolución que adopte el Fiscal por la que concluya las presentes diligencias de investigación’” a que se refiere el Apartado XI de la Circular 4/2013 de la Fiscalía General del Estado, interesándose asimismo que, judicializada sea la causa, se nos realice el correspondiente ofrecimiento de acciones del artículo 776 LECrim”."

# Ejemplo para Explicar
user_input = "¿Que pasos sigue una denuncia una vez presentada?"
result = explain_text(user_input, document_text)
print(f"Resultado: {result}")

# Ejemplo para Traducir
target_language = "francés"
result = translate(document_text, target_language)
print(f"Resultado: {result}")

# Ejemplo para Resumir
summary_length = "largo"
result = summarize(document_text, summary_length)
print(f"Resultado: {result}")

Y por último ejemplo se incluye una política de privacidad para verificar un correcto funcionamiento de toda la funcionalidad.

In [ ]:
# Ejemplo politica de privacidad
document_text="POLÍTICA DE PRIVACIDAD\nEl presente Política de Privacidad establece los términos en que  usa y protege la información que es proporcionada por sus usuarios al momento de utilizar su sitio web. Esta compañía está comprometida con la seguridad de los datos de sus usuarios. Cuando le pedimos llenar los campos de información personal con la cual usted pueda ser identificado, lo hacemos asegurando que sólo se empleará de acuerdo con los términos de este documento. Sin embargo esta Política de Privacidad puede cambiar con el tiempo o ser actualizada por lo que le recomendamos y enfatizamos revisar continuamente esta página para asegurarse que está de acuerdo con dichos cambios.\n\nInformación que es recogida\n\nNuestro sitio web podrá recoger información personal por ejemplo: Nombre,  información de contacto como  su dirección de correo electrónica e información demográfica. Así mismo cuando sea necesario podrá ser requerida información específica para procesar algún pedido o realizar una entrega o facturación.\n\nUso de la información recogida\n\nNuestro sitio web emplea la información con el fin de proporcionar el mejor servicio posible, particularmente para mantener un registro de usuarios, de pedidos en caso que aplique, y mejorar nuestros productos y servicios.  Es posible que sean enviados correos electrónicos periódicamente a través de nuestro sitio con ofertas especiales, nuevos productos y otra información publicitaria que consideremos relevante para usted o que pueda brindarle algún beneficio, estos correos electrónicos serán enviados a la dirección que usted proporcione y podrán ser cancelados en cualquier momento.\n\n está altamente comprometido para cumplir con el compromiso de mantener su información segura. Usamos los sistemas más avanzados y los actualizamos constantemente para asegurarnos que no exista ningún acceso no autorizado.\n\nCookies\n\nUna cookie se refiere a un fichero que es enviado con la finalidad de solicitar permiso para almacenarse en su ordenador, al aceptar dicho fichero se crea y la cookie sirve entonces para tener información respecto al tráfico web, y también facilita las futuras visitas a una web recurrente. Otra función que tienen las cookies es que con ellas las web pueden reconocerte individualmente y por tanto brindarte el mejor servicio personalizado de su web.\n\nNuestro sitio web emplea las cookies para poder identificar las páginas que son visitadas y su frecuencia. Esta información es empleada únicamente para análisis estadístico y después la información se elimina de forma permanente. Usted puede eliminar las cookies en cualquier momento desde su ordenador. Sin embargo las cookies ayudan a proporcionar un mejor servicio de los sitios web, estás no dan acceso a información de su ordenador ni de usted, a menos de que usted así lo quiera y la proporcione directamente . Usted puede aceptar o negar el uso de cookies, sin embargo la mayoría de navegadores aceptan cookies automáticamente pues sirve para tener un mejor servicio web. También usted puede cambiar la configuración de su ordenador para declinar las cookies. Si se declinan es posible que no pueda utilizar algunos de nuestros servicios.\n\nEnlaces a Terceros\n\nEste sitio web pudiera contener en laces a otros sitios que pudieran ser de su interés. Una vez que usted de clic en estos enlaces y abandone nuestra página, ya no tenemos control sobre al sitio al que es redirigido y por lo tanto no somos responsables de los términos o privacidad ni de la protección de sus datos en esos otros sitios terceros. Dichos sitios están sujetos a sus propias políticas de privacidad por lo cual es recomendable que los consulte para confirmar que usted está de acuerdo con estas.\n\nControl de su información personal\n\nEn cualquier momento usted puede restringir la recopilación o el uso de la información personal que es proporcionada a nuestro sitio web.  Cada vez que se le solicite rellenar un formulario, como el de alta de usuario, puede marcar o desmarcar la opción de recibir información por correo electrónico.  En caso de que haya marcado la opción de recibir nuestro boletín o publicidad usted puede cancelarla en cualquier momento.\n\nEsta compañía no venderá, cederá ni distribuirá la información personal que es recopilada sin su consentimiento, salvo que sea requerido por un juez con un orden judicial.\n\n Se reserva el derecho de cambiar los términos de la presente Política de Privacidad en cualquier momento."

# Ejemplo para Explicar
user_input = "¿Que es una politica privacidad y para que sirve?"
result = explain_text(user_input, document_text)
print(f"Resultado: {result}")

# Ejemplo para Traducir
target_language = "italiano"
result = translate(document_text, target_language)
print(f"Resultado: {result}")

# Ejemplo para Resumir
summary_length = "corto"
result = summarize(document_text, summary_length)
print(f"Resultado: {result}")

{'Texto clasificado como: ': 'politicas_de_privacidad'}
Clasificación no reconocida.
Resultado: {'context': 'POLÍTICA DE PRIVACIDAD\nEl presente Política de Privacidad establece los términos en que  usa y protege la información que es proporcionada por sus usuarios al momento de utilizar su sitio web. Esta compañía está comprometida con la seguridad de los datos de sus usuarios. Cuando le pedimos llenar los campos de información personal con la cual usted pueda ser identificado, lo hacemos asegurando que sólo se empleará de acuerdo con los términos de este documento. Sin embargo esta Política de Privacidad puede cambiar con el tiempo o ser actualizada por lo que le recomendamos y enfatizamos revisar continuamente esta página para asegurarse que está de acuerdo con dichos cambios.\n\nInformación que es recogida\n\nNuestro sitio web podrá recoger información personal por ejemplo: Nombre,  información de contacto como  su dirección de correo electrónica e información demográfica. Así mism

Y por último terminamos con un contrato.

In [ ]:
# Ejemplo politica de privacidad
document_text="Contrato de compraventa de un vehículo usado entre particulares\nEn ........................... a.......... de .................... de 20...... HORA: ………………………..\nVendedor:\nD. ……………………………….…….……………………., con N.I.F. nº …………….……….………, y domicilio en ……………………….…., calle de ………………….……………..\nComprador:\nD. ………………………….…………….……….…………, con N.I.F. nº ……………….………………, y domicilio en........................................…., calle de…………….……….…….Vehículo:\nMarca: ...........................\nMatrícula: ...........................\nNº de Bastidor: …………………………………..\nKilómetros:………………….Reunidos vendedor y comprador en la fecha del encabezamiento, manifiestan haber acordado formalizar en este documento CONTRATO DE COMPRAVENTA del vehículo automóvil que se especifica, en las siguientes\nCONICIONES\n1ª) El vendedor vende al comprador el vehículo de su propiedad anteriormente especificado por la cantidad de …………… euros, sin incluir los impuestos correspondientes, que serán a cargo del comprador.\n2ª) El vendedor declara que no pesa sobre el vehículo ninguna carga o gravamen ni impuesto, deuda o sanción pendientes de abono en la fecha de la firma de este contrato, comprometiéndose en caso contrario a regularizar tal situación a su exclusivo cargo.\n3ª) El vendedor se compromete a facilitar los distintos documentos relativos al vehículo, así como a firmar cuantos documentos aparte de éste sean necesarios para que el vehículo quede correctamente inscrito a nombre del comprador en los correspondientes organismos públicos, siendo todos los gastos a cargo del comprador.\n4ª) Una vez realizada la correspondiente transferencia en Tráfico, el vendedor entregará materialmente al comprador la posesión del vehículo, haciéndose el comprador cargo de cuantas responsabilidades puedan contraerse por la propiedad del vehículo y su tenencia y uso a partir de dicho momento de la entrega.\n5ª) El vehículo dispone de seguro en vigor hasta fecha de………………… y se encuentra al corriente respecto a las obligaciones derivadas de la ITV (Inspección Técnica de Vehículos).\n6ª) El comprador declara conocer el estado actual del vehículo, por lo que exime al vendedor de garantía por vicios o defectos que surjan con posterioridad a la entrega, salvo aquellos ocultos que tengan su origen en dolo o mala fe del vendedor.\n7ª) Para cualquier litigio que surja entre las partes de la interpretación o cumplimiento del presente contrato, éstas, con expresa renuncia al fuero que pudiera corresponderles, se someterán a los Juzgados y Tribunales de............................\nY para que así conste, firman el presente contrato de compraventa, por triplicado, en la fecha y lugar arriba indicados.\nFirma del vendedor Firma del comprador"

# Ejemplo para Explicar
user_input = "¿En este contrato el vendedor ofrece algun tipo de garantía sobre el vehiculo adquirido?"
result = explain_text(user_input, document_text)
print(f"Resultado: {result}")

# Ejemplo para Traducir
target_language = "portugués"
result = translate(document_text, target_language)
print(f"Resultado: {result}")

# Ejemplo para Resumir
summary_length = "medio"
result = summarize(document_text, summary_length)
print(f"Resultado: {result}")